<a href="https://colab.research.google.com/github/chrishuskey/DS-Unit-2-Linear-Models/blob/master/Chris_Huskey_Sprint_Challenge_DS_Unit_2_Sprint_1_Linear_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Linear Models Sprint Challenge

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

### Part 1, Classification
- 1.1. Do train/test split. Arrange data into X features matrix and y target vector
- 1.2. Use scikit-learn to fit a logistic regression model
- 1.3. Report classification metric: accuracy

### Part 2, Regression
- 2.1. Begin with baselines for regression
- 2.2. Do train/validate/test split
- 2.3. Arrange data into X features matrix and y target vector
- 2.4. Do one-hot encoding
- 2.5. Use scikit-learn to fit a linear regression or ridge regression model
- 2.6. Report validation MAE and $R^2$

### Stretch Goals, Regression
- Make at least 2 visualizations to explore relationships between features and target. You may use any visualization library
- Try at least 3 feature combinations. You may select features manually, or automatically
- Report validation MAE and $R^2$ for each feature combination you try
- Report test MAE and $R^2$ for your final model
- Print or plot the coefficients for the features in your model

In [0]:
# If you're in Colab...
import sys
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*

# Part 1, Classification: Predict Blood Donations 🚑
Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict whether the donor made a donation in March 2007, using information about each donor's history.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

In [0]:
import pandas as pd

donors = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')
assert donors.shape == (748,5)

donors = donors.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [0]:
# # Split into not_test and test sets:
from sklearn.model_selection import train_test_split

working_data, test = train_test_split(donors, test_size=0.20)

In [0]:
# Data prep.:  Function to do data prep. on each set:
def data_prep(dataframes):
  for dataframe in dataframes:
    dataframe['avg_donations_per_month'] = dataframe['number_of_donations'] / dataframe['months_since_first_donation']
    # What other data prep. to do?  TBH just not sure how to approach this problem yet -- need to think about it for a second, but no time left... 

In [0]:
# Run data prep. for each dataset, after split into working_data vs. test, 
# but before we split working_data into train and val matrices and target vectors:
data_prep([working_data, test])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
features = ['months_since_last_donation', 'number_of_donations', 'months_since_first_donation']  # 'months_since_last_donation', 'number_of_donations', 'months_since_first_donation', 'avg_donations_per_month'
target = 'made_donation_in_march_2007'

# Baseline:  Majority Classifier:

In [0]:
donors[target].value_counts()

0    570
1    178
Name: made_donation_in_march_2007, dtype: int64

In [0]:
majority_class = donors[target].mode()[0]  # == donors['made_donation_in_march_2007'].value_counts().idxmax()

In [0]:
# Baseline Accuracy and Recall:  Dummy Model (Majority Classifier) Accuracy:
y_true_dummy_classifier = working_data[target]
y_pred_dummy_classifier = [majority_class] * len(working_data[target])
print('Baseline:  Dummy Model Majority Classifier Performance:\n')
print(f'Accuracy: {accuracy_score(y_true_dummy_classifier, y_pred_dummy_classifier):.2f}')
print(f'Recall (TP / (TP + FN)): {recall_score(y_true_dummy_classifier, y_pred_dummy_classifier):.2f}\n')

Baseline:  Dummy Model Majority Classifier Performance:

Accuracy: 0.75
Recall (TP / (TP + FN)): 0.00



In [0]:
# Confusion matrix (normalized):
print('Confusion matrix (normalized) for reference:')
cm_dummy_classifier = pd.DataFrame(confusion_matrix(y_true_dummy_classifier, y_pred_dummy_classifier), index=['Actual F', 'Actual T'], columns=['Predicted F', 'Predicted T'])
assert round((cm_dummy_classifier / cm_dummy_classifier.sum().sum()).sum().sum(), 10) == 1.0
cm_dummy_classifier / cm_dummy_classifier.sum().sum()

Confusion matrix (normalized) for reference:


,Predicted F,Predicted T
Actual F,0.749164,0.0
Actual T,0.250836,0.0


## 1.1. Do train/test split. Arrange data into X features matrix and y target vector

Do these steps in either order.

Use scikit-learn's train/test split function to split randomly. (You can include 75% of the data in the train set, and hold out 25% for the test set, which is the default.)

In [0]:
# # Split into train and validate sets:
from sklearn.model_selection import train_test_split

X_working_data = pd.DataFrame(data=working_data[features])
y_working_data = working_data[target]

X_train, X_val, y_train, y_val = train_test_split(X_working_data, y_working_data, train_size=0.75)

# Check to make sure train + val + test sets add up to the whole dataset:
assert X_train.shape[0] + X_val.shape[0] + test.shape[0] == donors.shape[0]

## 1.2. Use scikit-learn to fit a logistic regression model

You may use any number of features

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

# 1. Import estimator class:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold

# 2. Instantiate the estimator class:
log_reg = LogisticRegressionCV(cv=StratifiedKFold(n_splits=5), scoring='accuracy', solver='lbfgs', n_jobs=-1, random_state=42)  # max_iter=100 if takes too long
# not converging, regardless of which solver I use...

# 3. Make X feature matrices and y target vectors:
# Did this above

# Encode / impute missing values / scale if needed:

# Encode:

# Impute:
# Not needed, because no missing/NaN values

# Scale:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 4. Fit the model to our training data:
log_reg.fit(X_train_scaled, y_train)

# Training set accuracy:
y_true_train = y_train
y_pred_train = log_reg.predict(X_train_scaled)
print('Model performance on training set:\n')
print(f'Training set accuracy: {accuracy_score(y_true_train, y_pred_train):.2f}')
print(f'Training set recall (TP / (TP + FN)): {recall_score(y_true_train, y_pred_train):.2f}\n')

Model performance on training set:

Training set accuracy: 0.77
Training set recall (TP / (TP + FN)): 0.19



In [0]:
# Confusion matrix (normalized):
print('Confusion matrix (normalized) for reference:')
cm_train = pd.DataFrame(confusion_matrix(y_true_train, y_pred_train), index=['Actual F', 'Actual T'], columns=['Predicted F', 'Predicted T'])
assert round((cm_train / cm_train.sum().sum()).sum().sum(), 10) == 1.0
cm_train / cm_train.sum().sum()

Confusion matrix (normalized) for reference:


,Predicted F,Predicted T
Actual F,0.723214,0.024554
Actual T,0.205357,0.046875


## 1.3. Report classification metric: accuracy

What is your model's accuracy on the test set?

Don't worry if your model doesn't beat the majority class baseline. That's okay!

_"The combination of some data and an aching desire for an answer does not ensure that a reasonable answer can be extracted from a given body of data."_ —[John Tukey](https://en.wikiquote.org/wiki/John_Tukey)

(Also, if we used recall score instead of accuracy score, then your model would almost certainly beat the baseline. We'll discuss how to choose and interpret evaluation metrics throughout this unit.)


In [0]:
# 5. Apply our model to new data:
y_true_val = y_val
y_pred_val = log_reg.predict(X_val_scaled)
print('Model performance on validation set:\n')
print(f'Validation set accuracy: {accuracy_score(y_true_val, y_pred_val):.2f}')
print(f'Training set recall (TP / (TP + FN)): {recall_score(y_true_val, y_pred_val):.2f}\n')

Model performance on validation set:

Validation set accuracy: 0.79
Training set recall (TP / (TP + FN)): 0.22



In [0]:
# Confusion matrix (normalized):
print('Confusion matrix (normalized) for reference:')
cm_val = pd.DataFrame(confusion_matrix(y_true_val, y_pred_val), index=['Actual F', 'Actual T'], columns=['Predicted F', 'Predicted T'])
assert round((cm_val / cm_val.sum().sum()).sum().sum(), 10) == 1.0
cm_val / cm_val.sum().sum()

Confusion matrix (normalized) for reference:


,Predicted F,Predicted T
Actual F,0.740000,0.013333
Actual T,0.193333,0.053333


# Part 2, Regression: Predict home prices in Ames, Iowa 🏠

You'll use historical housing data. ***There's a data dictionary at the bottom of the notebook.*** 

Run this code cell to load the dataset:





In [0]:
# Import libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [0]:
import pandas as pd
URL = 'https://drive.google.com/uc?export=download&id=1522WlEW6HFss36roD_Cd9nybqSuiVcCK'
homes = pd.read_csv(URL)
assert homes.shape == (2904, 47)

## 2.1. Begin with baselines

What is the Mean Absolute Error and R^2 score for a mean baseline? (You can get these estimated scores using all your data, before splitting it.)

In [0]:
# Baseline:  Dummy Mean Model (same as DummyRegressor(strategy='mean')):
y_true_dummy_mean = homes['SalePrice']
y_pred_dummy_mean = np.full(len(homes['SalePrice']), homes['SalePrice'].mean())

In [0]:
# Get error metrics for Dummy Mean Model:
print('Baseline:  Dummy Mean Model Error:\n')
print(f'Mean absolute error (MAE): {mean_absolute_error(y_true_dummy_mean, y_pred_dummy_mean):.1f}')
mse_dummy_mean = mean_squared_error(y_true_dummy_mean, y_pred_dummy_mean)
print(f'Mean squared error (MSE): {mse_dummy_mean:.1f}')
print(f'Root Mean Squared Error (RMSE): {sqrt(mse_dummy_mean):.1f}')
print(f'R^2 score: {r2_score(y_true_dummy_mean, y_pred_dummy_mean):.2f}\n')  # Should be 0 because y = mean line has no variance

Baseline:  Dummy Mean Model Error:

Mean absolute error (MAE): 58149.9
Mean squared error (MSE): 6366297661.5
Root Mean Squared Error (RMSE): 79789.1
R^2 score: 0.00



## 2.2. Do train/validate/test split

Train on houses sold in the years 2006 - 2008. (1,920 rows)

Validate on house sold in 2009. (644 rows)

Test on houses sold in 2010. (340 rows)

In [0]:
# Train/validate/test split:
train = homes[homes['Yr_Sold'] <= 2008]
val = homes[homes['Yr_Sold'] == 2009]
test = homes[homes['Yr_Sold'] == 2010]

# Check to make sure we included all observations from the original data set:
assert train.shape[0] + val.shape[0] + test.shape[0] == homes.Yr_Sold.value_counts(dropna=False).sum()

In [0]:
# Data prep.:  Function to do all our data prep. on each set:
def data_prep(dataframes):
  for dataframe in dataframes:
    dataframe['Years_Since_Renovated'] = dataframe['Yr_Sold'] - dataframe['Year_Remod/Add']  # ['Year_Remod/Add'] already includes ['Year_Built'] if no renovation after that

In [0]:
# Run function:
data_prep([train, val, test])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 2.3. Arrange data into X features matrix and y target vector

Select at least one numeric feature and at least one categorical feature.

Otherwise, you may choose whichever features and however many you want.

In [0]:

features = ['Overall_Cond', '1st_Flr_SF', 'Lot_Area', 'Gr_Liv_Area', 'Bedroom_AbvGr', 'Years_Since_Renovated', 'TotRms_AbvGrd', 'Functional']
target = 'SalePrice'

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

# # Start:
# 'Overall_Cond', 
# '1st_Flr_SF', 
# 'Lot_Area', 
# 'Gr_Liv_Area', 
# 'Bedroom_AbvGr', 
# 'Years_Since_Renovated' 
# 'TotRms_AbvGrd', 

# # Categorical variable to encode:
# 'Functional', 

# # TBD:
# 'Bldg_Type', 

# 'Full_Bath', 
# 'Half_Bath', 
# 'Bsmt_Full_Bath', 
# 'Bsmt_Half_Bath', 

# 'House_Style', 
# 'Kitchen_AbvGr', 

# 'Sale_Condition', 
# 'Sale_Type', 

## 2.4. Do one-hot encoding

Encode your categorical feature(s).

In [0]:
import category_encoders as ce

In [0]:
# Encode categorical variable: ['Functional']:
encoder = ce.OneHotEncoder(cols='Functional', use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)

## 2.5. Use scikit-learn to fit a linear regression or ridge regression model
Fit your model.

In [0]:
# 1. Import estimator class:
from sklearn.linear_model import LinearRegression

# 2. Instantiate the estimator class:
lin_reg = LinearRegression()

# 3. Make X feature matrices and y target vectors:
# Did this above

# Encode / impute missing values / scale if needed:
# Encoded above already
# No need to impute, because no missing/NaN values

# 4. Fit the model to our training data:
lin_reg.fit(X_train_encoded, y_train)

# Training set error:
# Error on training set:
y_true_train = y_train
y_pred_train = lin_reg.predict(X_train_encoded)
print('Training Set: Model Error:')
print(f'MAE: {mean_absolute_error(y_true_train, y_pred_train):.1f}')
mse_train = mean_squared_error(y_true_train, y_pred_train)
print(f'MSE: {mse_train:.1f}')
print(f'RMSE: {sqrt(mse_train):.1f}')
print(f'R^2 score: {r2_score(y_true_train, y_pred_train):.2f}\n')

Training Set: Model Error:
MAE: 28052.4
MSE: 1969842594.3
RMSE: 44382.9
R^2 score: 0.69



## 2.6. Report validation MAE and $R^2$

What is your model's Mean Absolute Error and $R^2$ score on the validation set? (You are not graded on how high or low your validation scores are.)

In [0]:
# 5. Apply our model to new data:

# Error on validation set:
y_true_val = y_val
y_pred_val = lin_reg.predict(X_val_encoded)
print('Validation Set Error:')
print(f'MAE: {mean_absolute_error(y_true_val, y_pred_val):.1f}')
mse_val = mean_squared_error(y_true_val, y_pred_val)
print(f'MSE: {mse_val:.1f}')
print(f'RMSE: {sqrt(mse_val):.1f}')
print(f'R^2 score: {r2_score(y_true_val, y_pred_val):.2f}')

Validation Set Error:
MAE: 28336.2
MSE: 1694441822.3
RMSE: 41163.6
R^2 score: 0.74


# Stretch Goals, Regression
- Make at least 2 visualizations to explore relationships between features and target. You may use any visualization library
- Try at least 3 feature combinations. You may select features manually, or automatically
- Report validation MAE and $R^2$ for each feature combination you try
- Report test MAE and $R^2$ for your final model
- Print or plot the coefficients for the features in your model

# Data Dictionary 

Here's a description of the data fields:

```
1st_Flr_SF: First Floor square feet

Bedroom_AbvGr: Bedrooms above grade (does NOT include basement bedrooms)

Bldg_Type: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit
       
Bsmt_Half_Bath: Basement half bathrooms

Bsmt_Full_Bath: Basement full bathrooms

Central_Air: Central air conditioning

       N	No
       Y	Yes
		
Condition_1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
Condition_2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
       
 Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed
       
 Exter_Cond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
 
 Exter_Qual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
Exterior_1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
Exterior_2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
       
Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood
		
Full_Bath: Full bathrooms above grade

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only
		
Gr_Liv_Area: Above grade (ground) living area square feet
        
Half_Bath: Half baths above grade

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace
		
Heating_QC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

House_Style: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level

Kitchen_AbvGr: Kitchens above grade

Kitchen_Qual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor

LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression
		
Land_Slope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope

Lot_Area: Lot size in square feet

Lot_Config: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property

Lot_Shape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular

MS_SubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MS_Zoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density

Mas_Vnr_Type: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone

Mo_Sold: Month Sold (MM)

Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker
			
Overall_Cond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor

Overall_Qual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

Paved_Drive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel

Roof_Matl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles

Roof_Style: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed

SalePrice: the sales price for each house

Sale_Condition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)

Sale_Type: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
	
Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved
       	
TotRms_AbvGrd: Total rooms above grade (does not include bathrooms)

Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	
	
Year_Built: Original construction date

Year_Remod/Add: Remodel date (same as construction date if no remodeling or additions)
						
Yr_Sold: Year Sold (YYYY)	

```